# Account Period Data for a BSC Party from a SAA-I014 Data file

**Author:** Muhammad Raees Usman

**Created Date:** 31/10/2022  

This simple script retrieves the Account Period Data for a single BSC Party and Settlement Date using a SAA-I014 text file as an input. The script is intended to be used as an example of how the SAA-I014 file can be imported and manipulated in Python using Pandas given the complex nested structure of the data. The manipulations completed below are in line with the Interface Design Document structure format of the data flow. A link to this file can be found in the accompanying Insight Article.  

In [ ]:
# The relevant Python modules are imported
import numpy as np
import pandas as pd

In [ ]:
# A user input is requested for the data file location and BSC Party Id
# An example input could be: C:\Users\Name\Desktop\S0142_20220815_SF_20220907113309.txt
input_loc = input('Please enter the file location:')

# User input of the desired BSC Party Id for which data is retrieved
bsc_party_id = input('Please enter the BSC Party Id:')

The file conatins rows that have differing number of columns. Therefore, a DataFrame needs to be created that matches the maximum size of the data file in terms of rows and columns. This is achieved in the code block below.

In [ ]:
# Opening the data file
with open(input_loc, 'r') as temp_data:
    # Retrive the No. of columns for each row in the file
    col_count = [len(l.split("|")) for l in temp_data.readlines()]
    
# Create the column names from 0 - (maximum column No. -1)
column_names = [i for i in range(0, max(col_count))]

# Import the data into a Pandas DataFrame
df = pd.read_csv(input_loc, header=None, delimiter="|", names=column_names)

The code block below looks through the DataFrame and retrieves data for the user inputted BSC Party Id. This is achieved by using the headers in the data structure file to identify which rows in the file initiate the nested data for a specific BSC Party ID and Settlement Period. 

In [ ]:
low_lim = 0 # Initiate low limit variable

# Loop through each row in the DataFrame
for i, line in df.iterrows(): 
    if line[0] == 'BPH':            # Identify the BSC Party through the BSC Party Header (BPH) rows in the file
        if line[8] == bsc_party_id: # Check if the row corresponds to the user inputted BSC Party Id
            low_lim = i             # Assign a low limit for the row No. for which the data file will be iterated

df1 = pd.DataFrame()                # Create an empty DataFrame to store the data
df1['Settlement Period'] = 0        # Add a Settlement Period column

#Loop through all nested rows for the BSC Party Id using the low row limit
for i, line in df[low_lim+1:].iterrows():
    if line[0] == 'SP7':        # Identify the Settlement Period
        SP = line[1]
    if line[0] == 'APD':        # Identify the APD for the Settlement Period
        df1 = df1.append(line)
        df1.loc[i, 'Settlement Period'] = SP
    if line[0] == 'BPH':        # Exit loop once next BPH has been detected
        break
        
settlement_date = df.loc[1,1]             # Identify Settlement Period for csv file name
run_type = df.loc[1,2]                    # Identify Settlement Run Type
df1['Settlement Date'] = settlement_date[6:8] + '/' + settlement_date[4:6] + '/' + settlement_date[0:4] # Add Settlement Date to DataFrame
df1['Settlement Run Type'] = run_type     # Add Settlement Run Type to DataFrame
        
df_final = df1.loc[:,['Settlement Date','Settlement Period','Settlement Run Type',1,2,3,4,5,6,7,8,9,10]] # Extract relevant columns of interest

# Rename column in line with data file IDD
df_final.rename(columns={1: 'Production/Consumption Flag', 2: 'BSSC Limited Cost Allocation', 3: 'Energy Imbalance Charge', 4: 'Information Imbalance Charge', 5: 'Residual Cashflow Reallocation Charge', 6: 'Account Bilateral Contract Volume', 7: 'Account Period Balancing Services Volume', 8: 'Account Energy Imbalance Volume', 9: 'Account Credited Energy Volume', 10: 'Residual Cashflow Reallocation Proportion'}, inplace=True)
df_final.reset_index(drop=True, inplace=True) # Reset Index of DataFrame
df_final.to_csv('APD_' + bsc_party_id + '_' +  settlement_date + '_' + run_type + '.csv', index=False) # Export data to a csv file
df_final # Display the Dataframe inline for reference